In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://calendar.carleton.ca/undergrad/undergradprograms/engineering/"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

In [5]:
page_soup = soup(driver.page_source, 'lxml')

# 1. Collect link texts for each course (for driver to click on)

In [6]:
table = page_soup.find("h3", {"id": "Mechanical_Engineering__Bachelor_of_Engineering"}).findNextSibling()
table

<div class="table-container"><table class="sc_courselist" width="100%"><colgroup><col class="codecol"/><col class="titlecol"/><col class="hourscol"/></colgroup><tbody><tr class="even areaheader firstrow"><td colspan="2"><span class="courselistcomment areaheader">First year</span></td><td class="hourscol"></td></tr> <tr class="odd areaheader"><td colspan="2"><span class="courselistcomment areaheader">1. a) 4.0 credits in:</span></td><td class="hourscol">4.0</td></tr> <tr class="even"><td class="codecol"><div style="margin-left: 20px;"><a class="bubblelink code" href="/search/?P=CHEM%201101" onclick="return showCourse(this, 'CHEM 1101');" title="CHEM 1101">CHEM 1101</a> [0.5]</div></td><td>Chemistry for Engineering Students</td><td class="hourscol"></td></tr> <tr class="odd"><td class="codecol"><div style="margin-left: 20px;"><a class="bubblelink code" href="/search/?P=ECOR%201041" onclick="return showCourse(this, 'ECOR 1041');" title="ECOR 1041">ECOR 1041</a> [0.25]</div></td><td>Comput

In [7]:
containers = table.findAll("a", {"class": "bubblelink code"})
len(containers)

44

In [8]:
link_texts = [container.text.replace("\xa0", " ") for container in containers]
link_texts

['CHEM 1101',
 'ECOR 1041',
 'ECOR 1042',
 'ECOR 1043',
 'ECOR 1044',
 'ECOR 1045',
 'ECOR 1046',
 'ECOR 1047',
 'ECOR 1048',
 'MATH 1004',
 'MATH 1104',
 'PHYS 1004',
 'ECOR 1055',
 'ECOR 1056',
 'ECOR 1057',
 'ECOR 2050',
 'ELEC 3605',
 'MAAE 2001',
 'MAAE 2101',
 'MAAE 2202',
 'MAAE 2300',
 'MAAE 2400',
 'MAAE 2700',
 'MATH 1005',
 'MATH 2004',
 'ECOR 2995',
 'CCDP 2100',
 'ECOR 3800',
 'MAAE 3004',
 'MAAE 3202',
 'MAAE 3300',
 'MAAE 3400',
 'MAAE 3500',
 'MATH 3705',
 'MECH 3002',
 'MECH 3700',
 'SYSC 3600',
 'ECOR 4995',
 'MAAE 4102',
 'MAAE 4907',
 'MECH 4003',
 'MECH 4406',
 'ELEC 4504',
 'ELEC 4602']

# 2. Test run - scrape the info of the first course

In [9]:
driver.find_element_by_link_text(link_texts[0]).click()

In [10]:
page_soup = soup(driver.page_source, 'lxml')

In [11]:
container = page_soup.find("div", {"class": "lfjsbubble"}).find("div", {"class": "courseblock"})

In [12]:
course_code = container.find("span", {"class": "courseblockcode"}).text.replace("\xa0", " ").strip()
course_code

'CHEM 1101'

In [13]:
course_name = container.find("span", {"class": "courseblocktitle"}).text.split("\n")[1].strip()
course_name

'Chemistry for Engineering Students'

In [14]:
course_desc = container.strong.next_sibling.next_sibling.strip()
course_desc

'Topics include stoichiometry, atomic and molecular structure, thermodynamics and chemical equilibrium, acid-base chemistry, carbon dioxide in water, alkalinity, precipitation, electrochemistry, kinetics and basic organic chemistry. Laboratory component emphasizes techniques and methods of basic experimental chemistry.'

In [15]:
from selenium.webdriver.common.keys import Keys

driver.find_element_by_link_text(link_texts[0]).send_keys(Keys.ESCAPE)

# 3. Test run successful! Implement automation script to scrape all courses

In [16]:
counter = 0
course_codes = []
course_names = []
course_descs = []

for link_text in link_texts:
    
    link = driver.find_element_by_link_text(link_text)
    time.sleep(2)
    
    link.click()
    time.sleep(3)
    
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("div", {"class": "lfjsbubble"}).find("div", {"class": "courseblock"})
    
    course_code = container.find("span", {"class": "courseblockcode"}).text.replace("\xa0", " ").strip()
    course_name = container.find("span", {"class": "courseblocktitle"}).text.split("\n")[1].strip()
    course_desc = container.strong.next_sibling.next_sibling.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    link.send_keys(Keys.ESCAPE)
    time.sleep(2)
    
print("scraped {} courses".format(counter))

Scraped  CHEM 1101
Scraped  ECOR 1041
Scraped  ECOR 1042
Scraped  ECOR 1043
Scraped  ECOR 1044
Scraped  ECOR 1045
Scraped  ECOR 1046
Scraped  ECOR 1047
Scraped  ECOR 1048
Scraped  MATH 1004
Scraped  MATH 1104
Scraped  PHYS 1004
Scraped  ECOR 1055
Scraped  ECOR 1056
Scraped  ECOR 1057
Scraped  ECOR 2050
Scraped  ELEC 3605
Scraped  MAAE 2001
Scraped  MAAE 2101
Scraped  MAAE 2202
Scraped  MAAE 2300
Scraped  MAAE 2400
Scraped  MAAE 2700
Scraped  MATH 1005
Scraped  MATH 2004
Scraped  ECOR 2995
Scraped  CCDP 2100
Scraped  ECOR 3800
Scraped  MAAE 3004
Scraped  MAAE 3202
Scraped  MAAE 3300
Scraped  MAAE 3400
Scraped  MAAE 3500
Scraped  MATH 3705
Scraped  MECH 3002
Scraped  MECH 3700
Scraped  SYSC 3600
Scraped  ECOR 4995
Scraped  MAAE 4102
Scraped  MAAE 4907
Scraped  MECH 4003
Scraped  MECH 4406
Scraped  ELEC 4504
Scraped  ELEC 4602
scraped 44 courses


# 4. Inspect scraped data and write to csv

In [17]:
len(list(dict.fromkeys(course_codes)))

44

In [18]:
course_codes

['CHEM 1101',
 'ECOR 1041',
 'ECOR 1042',
 'ECOR 1043',
 'ECOR 1044',
 'ECOR 1045',
 'ECOR 1046',
 'ECOR 1047',
 'ECOR 1048',
 'MATH 1004',
 'MATH 1104',
 'PHYS 1004',
 'ECOR 1055',
 'ECOR 1056',
 'ECOR 1057',
 'ECOR 2050',
 'ELEC 3605',
 'MAAE 2001',
 'MAAE 2101',
 'MAAE 2202',
 'MAAE 2300',
 'MAAE 2400',
 'MAAE 2700',
 'MATH 1005',
 'MATH 2004',
 'ECOR 2995',
 'CCDP 2100',
 'ECOR 3800',
 'MAAE 3004',
 'MAAE 3202',
 'MAAE 3300',
 'MAAE 3400',
 'MAAE 3500',
 'MATH 3705',
 'MECH 3002',
 'MECH 3700',
 'SYSC 3600',
 'ECOR 4995',
 'MAAE 4102',
 'MAAE 4907',
 'MECH 4003',
 'MECH 4406',
 'ELEC 4504',
 'ELEC 4602']

In [19]:
course_names

['Chemistry for Engineering Students',
 'Computation and Programming',
 'Data Management',
 'Circuits',
 'Mechatronics',
 'Statics',
 'Mechanics',
 'Visual Communication',
 'Dynamics',
 'Calculus for Engineering or Physics',
 'Linear Algebra for Engineering or Science',
 'Introductory Electromagnetism and Wave Motion',
 'Introduction to Engineering Disciplines I',
 'Introduction to Engineering Disciplines II',
 'Engineering Profession',
 'Design and Analysis of Engineering Experiments',
 'Electrical Engineering',
 'Engineering Graphical Design',
 'Engineering Dynamics',
 'Mechanics of Solids I',
 'Fluid Mechanics I',
 'Thermodynamics and Heat Transfer',
 'Engineering Materials',
 'Differential Equations and Infinite Series for Engineering or Physics',
 'Multivariable Calculus for Engineering or Physics',
 'Engineering Portfolio',
 'Communication Skills for Engineering Students',
 'Engineering Economics',
 'Dynamics of Machinery',
 'Mechanics of Solids II',
 'Fluid Mechanics II',
 'Appl

In [20]:
course_descs

['Topics include stoichiometry, atomic and molecular structure, thermodynamics and chemical equilibrium, acid-base chemistry, carbon dioxide in water, alkalinity, precipitation, electrochemistry, kinetics and basic organic chemistry. Laboratory component emphasizes techniques and methods of basic experimental chemistry.',
 'Software development as an engineering discipline, using a modern programming language. Language syntax and semantics. Tracing and visualizing program execution. Program style and documentation. Testing and debugging tools and techniques.',
 'Introduction to designing and implementing numerical algorithms. Container data types: sequences, sets, maps. Modules. Data files. Incremental, iterative development of programs. Number systems: binary, decimal. Digital representation of integers and floating point numbers.',
 "Electrical Quantities (Voltage, Charge, Current, Power). Conservation of charge and energy. Mathematical models of simple devices. Elementary circuit th

In [21]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,CHEM 1101,Chemistry for Engineering Students,"Topics include stoichiometry, atomic and molec..."
1,ECOR 1041,Computation and Programming,Software development as an engineering discipl...
2,ECOR 1042,Data Management,Introduction to designing and implementing num...
3,ECOR 1043,Circuits,"Electrical Quantities (Voltage, Charge, Curren..."
4,ECOR 1044,Mechatronics,Mechatronics applications. Analog to digital s...
5,ECOR 1045,Statics,Cartesian vector representation of forces. Com...
6,ECOR 1046,Mechanics,2D truss analysis (method of joints/sections)....
7,ECOR 1047,Visual Communication,"Graphs and sketches, flow charts, block diagra..."
8,ECOR 1048,Dynamics,Kinematics and kinetics of a particle. Princip...
9,MATH 1004,Calculus for Engineering or Physics,Limits. Differentiation of the elementary func...


In [22]:
df.iloc[3, 2]

"Electrical Quantities (Voltage, Charge, Current, Power). Conservation of charge and energy. Mathematical models of simple devices. Elementary circuit theory for passive elements. Thévenin's and superposition theorem. Signal filtering and amplification. Time and frequency domain. Circuit design and simulation."

In [23]:
df.to_csv('Carleton_MechEng_Core_(AllYears)_Courses.csv', index = False)

In [24]:
driver.quit()